<a href="https://www.kaggle.com/code/rubanzasilva/symptom-disease?scriptVersionId=229811557" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction: Predictive model for differential diagnosis

In this notebook, our goal is to develop a model that can take in a patient's symptoms as an input and return a list of the top 3 possible classes (diseases) alongside confidence values for each class expressed as probabilities.


## Library and Data import

In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/symptoms-disease-no-id/symptom_disease_no_id_col.csv
/kaggle/input/symptoms-disease-no-id/symptom_no_id.csv


In [2]:
%%time
%pip install catboost
%pip install optuna
#%pip install optuna_distributed
#%pip install openfe
%pip install seaborn
%pip install xgboost
%pip install lightgbm
%pip install fastkaggle
#%pip install h2o
%pip install -Uqq fastbook
#%pip install polars
#%pip install -q -U autogluon.tabular
#%pip install autogluon
%pip install --upgrade pip
%pip install tqdm
#%pip install wandb
#%pip install sweetviz

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 52.8 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.0
    Uninstalling pip-24.0:
      Successfully uninstalled pip-24.0
Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.
CPU times: user 811 ms, sys: 273 ms, total: 1.08 s
Wall time: 1min 21s


In [3]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm

#import fastbook
#fastbook.setup_book()
#from fastbook import *
from fastai.tabular.all import *
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from numpy import random
from tqdm import tqdm
from ipywidgets import interact

from fastai.imports import *
np.set_printoptions(linewidth=130)
from fastai.text.all import *


from sklearn.ensemble import RandomForestClassifier
#from sklearn.metrics import root_mean_squared_error
from sklearn.metrics import accuracy_score
from sklearn.ensemble import VotingClassifier,StackingClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import KFold,StratifiedKFold, cross_val_score,train_test_split,GridSearchCV


from pathlib import Path
import os

import xgboost as xgb
from xgboost import plot_importance
from xgboost import XGBClassifier

import lightgbm as lgb
from lightgbm import LGBMClassifier

from catboost import CatBoostClassifier,CatBoostRegressor,Pool, metrics, cv


import warnings


#from openfe import OpenFE, transform
#from autogluon.tabular import TabularDataset, TabularPredictor

#import h2o
#from h2o.automl import H2OAutoML

import gc

import optuna
from optuna.samplers import TPESampler

import pickle
from joblib import dump, load

#import sweetviz as sv
#from IPython.display import FileLink

#import h2o
#from h2o.automl import H2OAutoML

Possible Approaches

1. Using the fastai text transfer learning approach which employs a language model (A model trained to predict the next text in a sentence).
   This employs self-supervised learning, to quote the fastai book
   > Self-supervised learning: Training a model using labels that are embedded in the independent variable, rather than requiring external labels. For instance, training a model to predict the next word in a text.
3. Using traditional tabular models and implementations such as xgboost, to do this we would have to convert our text values (symptoms) to numerical features.

In [4]:
path = Path('/kaggle/input/symptoms-disease-no-id')
path

Path('/kaggle/input/symptoms-disease-no-id')

# Exploratory Data Analysis

In [5]:
#symptom_df = pd.read_csv(path_lm/'symptom_synth.csv',index_col=0)
symptom_df = pd.read_csv(path/'symptom_no_id.csv')
sd_df = pd.read_csv(path/'symptom_disease_no_id_col.csv')
symptom_df.head()

,text
0,"I have been experiencing a skin rash on my arms, legs, and torso for the past few weeks. It is red, itchy, and covered in dry, scaly patches."
1,"My skin has been peeling, especially on my knees, elbows, and scalp. This peeling is often accompanied by a burning or stinging sensation."
2,"I have been experiencing joint pain in my fingers, wrists, and knees. The pain is often achy and throbbing, and it gets worse when I move my joints."
3,"There is a silver like dusting on my skin, especially on my lower back and scalp. This dusting is made up of small scales that flake off easily when I scratch them."
4,"My nails have small dents or pits in them, and they often feel inflammatory and tender to the touch. Even there are minor rashes on my arms."


In [6]:
df = sd_df

In [7]:
df.shape

(1200, 2)

In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   label   1200 non-null   object
 1   text    1200 non-null   object
dtypes: object(2)
memory usage: 18.9+ KB


In [9]:
#Missing values returned as percentages.
missing_percentages = df.isnull().mean() * 100
sorted_missing_percentages = missing_percentages.sort_values(ascending=False)
print(sorted_missing_percentages)

label    0.0
text     0.0
dtype: float64


In [10]:
df.describe().T

,count,unique,top,freq
label,1200,24,Psoriasis,50
text,1200,1153,"I've been feeling extremely scratchy, sick, and worn out. In addition, I've lost weight and have a temperature. My urine is dark, and my skin has turned yellow. Additionally, I have been experiencing stomach pain.",4


So there are 24 diseases in this dataset

In [11]:
#X = df['label']
#y = df['text']

In [12]:
#X_train, X_test, y_train, y_test = train_test_split(
    #X, y, test_size=0.20, random_state=42)

In [13]:
#splits = RandomSplitter(valid_pct=0.2)(range_of(df))

# Approach 1 - Text transfer learning
Train a text classifier from a pretrained model


## Using the high-level API

In [14]:
dls = TextDataLoaders.from_df(df, path=path, text_col='text', label_col='label',valid_pct=0.2)

In [15]:
dls.show_batch()

,text,category
0,"xxbos xxmaj i 've had this cough that 's been lingering for days and difficulty breathing . xxmaj my fever is xxunk - high , and xxmaj i 'm feeling so weak and tired . xxmaj i 've also been producing a lot of mucus when i cough , and it 's just been so overwhelming to deal with all of these symptoms . xxmaj i 'm getting so xxunk with all of this .",Bronchial Asthma
1,"xxbos xxmaj mom , xxmaj xxunk , xxmaj i 've been feeling really tired and weak lately , and xxmaj i 've had this cough that just wo n't go away . xxmaj it 's been hard for me to catch my breath , and my fever has been really high . xxmaj when i cough , xxmaj i 've been producing a lot of mucus .",Bronchial Asthma
2,"xxbos xxmaj i 've been feeling really ill lately . xxmaj i 've had this persistent cough and difficulty breathing , and my fever has been off the xxunk . xxmaj i 'm also feeling extremely exhausted , and xxmaj i 've been producing a lot of mucus when i cough . xxmaj it 's just been so overwhelming to deal with all of these symptoms",Bronchial Asthma
3,xxbos xxmaj my eyes are red and watery all the time . xxmaj i 've also had this pressure in my sinuses that wo n't go away . xxmaj i 'm always feeling tired and xxmaj i 've been having a lot of trouble breathing . xxmaj i 've also had a lot of gunk in my throat and my lymph nodes are swollen .,Common Cold
4,"xxbos xxmaj i 've been struggling with difficulty breathing , a constant cough , and fatigue . xxmaj my fever is extremely high , and xxmaj i 've been coughing up a lot of thick , mucoid sputum . xxmaj it 's been so hard to deal with all of this , and xxmaj i 'm just feeling so drained and worn out",Bronchial Asthma
5,xxbos xxmaj i 've been having a really hard time going to the bathroom lately . xxmaj it 's really painful and xxmaj i 'm only able to go every few days . xxmaj there 's also a lot of pain in my anus and around that area . xxmaj my stool has been really bloody and my anus feels really irritated .,Dimorphic Hemorrhoids
6,"xxbos xxmaj i 've been having a tough time with this constant cough , difficulty breathing , and fatigue . xxmaj my fever is high , and xxmaj i 've been coughing up a lot of thick , mucoid sputum . xxmaj it 's all been rather xxunk and exhausting , and xxmaj i 'm feeling quite sick at the xxunk .",Bronchial Asthma
7,"xxbos xxmaj recently , xxmaj i 've been feeling terrible . xxmaj my fever has been through the roof , and xxmaj i 've had a persistent cough and breathing problems . xxmaj i 've been coughing up a lot of phlegm and am also feeling quite xxunk . xxmaj dealing with all of these symptoms has been quite difficult .",Bronchial Asthma
8,"xxbos xxmaj recently , xxmaj i 've felt quite weak and exhausted , and xxmaj i 've also had a cough that simply wo n't go away . xxmaj my fever has been really high , and it has been challenging for me to catch my breath . xxmaj i 've been making a lot of mucous when i cough .",Bronchial Asthma


> the library automatically processed all the texts to split then in tokens, adding some special tokens like:

xxbos to indicate the beginning of a te
t
xxmaj to indicate the next word was capitalized

In [16]:
learn = text_classifier_learner(dls, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

/opt/conda/lib/python3.10/site-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [17]:
%%time
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.791163,2.570458,0.545833,00:01


epoch,train_loss,valid_loss,accuracy,time
0,1.807653,1.806998,0.758333,00:01
1,1.485285,0.887681,0.820833,00:01
2,1.152863,0.498840,0.900000,00:01
3,0.905848,0.451232,0.920833,00:01


CPU times: user 5.76 s, sys: 1.38 s, total: 7.13 s
Wall time: 9.08 s


In [18]:
%%time
learn.fine_tune(4, 1e-2)

epoch,train_loss,valid_loss,accuracy,time
0,0.480280,0.362858,0.891667,00:01


epoch,train_loss,valid_loss,accuracy,time
0,0.356266,0.245201,0.950000,00:01
1,0.289745,0.211668,0.945833,00:01
2,0.226286,0.174830,0.962500,00:01
3,0.174960,0.189918,0.945833,00:01


CPU times: user 5.43 s, sys: 976 ms, total: 6.41 s
Wall time: 8.2 s


In [19]:
learn.show_results()

,text,category,category_
0,"xxbos xxmaj i 've had a persistent cough for days , and i also have breathing problems . xxmaj i 'm so weak and exhausted , and my fever is through the roof . xxmaj it 's been very difficult to xxunk all of these symptoms , and xxmaj i 've also been coughing up a lot of mucus . xxmaj i 'm becoming so xxunk over everything .",Bronchial Asthma,Bronchial Asthma
1,"xxbos xxunk xxunk that xxmaj i 've been experiencing shortness of breath , a persistent cough , and a high fever . xxmaj i 've also been producing a lot of thick , mucoid sputum , and xxmaj i 'm feeling quite drained and exhausted . xxmaj it 's all been rather tough to deal with , and xxmaj i 'm feeling quite sick at the xxunk",Bronchial Asthma,Bronchial Asthma
2,"xxbos xxmaj my eyes are always red and itchy , and my nose feels all stuffy and congested . i just feel kind of sick and tired all the time , and i keep coughing up all this gunk . xxmaj my throat feels sore and scratchy , and xxmaj i 've noticed that the bumps on my neck are xxunk than usual",Common Cold,Common Cold
3,"xxbos xxmaj i 've been having xxunk and loose , watery stools many times a day . xxmaj i 've lost my appetite and am always sick . xxmaj i 'm also developing a mild fever . xxmaj also , my abdominal part pains a lot . i do n't know what the reason behind all of these is .",Typhoid,Typhoid
4,"xxbos i have a lot of difficulty breathing . i do n't feel well , and xxmaj i 'm perspiring a lot . xxmaj my chest hurts , and i have a lot of mucus in my throat . xxmaj i 'm having trouble breathing , and the phlegm xxmaj i 'm coughing up has a strange hue .",Pneumonia,Pneumonia
5,"xxbos xxmaj i 've been experiencing a lot of bowel movement issues recently . xxmaj it 's difficult to go , and it hurts when i do . xxmaj my anus is quite painful , and it has been bleeding whenever i go . xxmaj it 's excruciatingly painful , and xxmaj i 'm quite uneasy .",Dimorphic Hemorrhoids,Dimorphic Hemorrhoids
6,"xxbos xxmaj i 've been struggling with a persistent cough , breathing issues , and exhaustion . xxmaj i 've been coughing up a lot of thick , mucoid sputum and have a high fever . xxmaj all of this has been really xxunk and exhausting , and right now xxmaj i 'm feeling rather ill .",Bronchial Asthma,Bronchial Asthma
7,xxbos xxmaj i 've been having a lot of trouble with my bowel movements lately . xxmaj it 's hard to go and it hurts when i do . xxmaj my anus is really sore and it 's been bleeding when i go . xxmaj it 's really painful and xxmaj i 'm really uncomfortable .,Dimorphic Hemorrhoids,Dimorphic Hemorrhoids
8,"xxbos xxmaj i 've been experiencing severe weariness and a xxunk sensation . xxmaj my throat has been really painful and xxmaj i 've had a fairly severe cough . xxmaj my nose has been quite stuffy , and xxmaj i 've had significant sinus pressure . xxmaj i 'm really miserable and extremely exhausted .",Common Cold,Common Cold


### Inference

In [20]:
learn.predict("I am having a running stomach, fever, general body weakness and have been getting bitten by mosquitoes often")

('gastroesophageal reflux disease',
 tensor(21),
 tensor([0.0054, 0.0073, 0.2410, 0.0201, 0.0004, 0.0018, 0.0011, 0.0091, 0.0122,
         0.1181, 0.0025, 0.0048, 0.0032, 0.1512, 0.0551, 0.0017, 0.0169, 0.0005,
         0.0204, 0.0404, 0.0024, 0.2430, 0.0049, 0.0364]))

In [21]:
learn.predict("xxbos i have a lot of itching all over my skin , and sometimes it gets really red and bumpy . xxmaj there are also some patches that are different colors than the rest of my skin , and sometimes i get these little bumps that look like little balls . xxmaj it 's really annoying and itchy.Fungal ")

('Fungal infection',
 tensor(8),
 tensor([1.0650e-03, 2.3092e-04, 3.8649e-05, 1.1999e-05, 3.6725e-02, 3.8615e-04,
         1.8858e-05, 5.9246e-04, 9.3193e-01, 1.5544e-05, 3.5409e-04, 1.3711e-04,
         3.4111e-04, 3.9881e-05, 5.6569e-06, 1.4484e-03, 3.2398e-04, 4.2299e-04,
         1.0479e-02, 1.5804e-03, 1.2884e-02, 3.7053e-04, 3.4867e-04, 2.5350e-04]))

In [22]:
#malaria_df = df[df['label'] == 'Malaria']
#malaria_df.head()

### Trying to return probabilities

In [23]:
print(len(learn.dls.vocab))

2


In [24]:
#print(learn.dls.vocab)

In [25]:
def get_top_3_predictions(text, learn):
    # Get prediction and probabilities
    _, _, probs = learn.predict(text)
    
    # Get the disease labels vocabulary (second list in vocab)
    disease_vocab = learn.dls.vocab[1]  # Access the disease labels
    
    # Get number of classes
    n_classes = len(disease_vocab)
    
    # Get indices of top 3 (or fewer) probabilities
    n_preds = min(3, n_classes)
    top_k_indices = probs.argsort(descending=True)[:n_preds]
    
    # Get the actual labels and their probabilities
    predictions = []
    for idx in top_k_indices:
        label = disease_vocab[int(idx)]  # Use disease_vocab instead of vocab
        probability = float(probs[idx])
        predictions.append((label, f"{probability:.3f}"))
    
    return predictions

def predict_batch_top_3(texts, learn):
    predictions = []
    for text in texts:
        preds = get_top_3_predictions(text, learn)
        predictions.append(preds)
    return predictions

# Test with a single prediction first
test_text = "severe headache with light sensitivity"
single_pred = get_top_3_predictions(test_text, learn)
print("\nSingle prediction test:")
for i, (disease, prob) in enumerate(single_pred, 1):
    print(f"{i}. {disease}: {prob}")

# Then test batch predictions
test_texts = ["severe headache with light sensitivity", 
              "itchy red rash with scaling",
              "frequent urination and excessive thirst"]

results = predict_batch_top_3(test_texts, learn)

print("\nBatch predictions:")
for i, preds in enumerate(results):
    print(f"\nText {i+1}:")
    for j, (disease, prob) in enumerate(preds, 1):
        print(f"{j}. {disease}: {prob}")


Single prediction test:
1. Migraine: 0.510
2. Pneumonia: 0.151
3. diabetes: 0.136



Batch predictions:

Text 1:
1. Migraine: 0.510
2. Pneumonia: 0.151
3. diabetes: 0.136

Text 2:
1. Migraine: 0.676
2. Pneumonia: 0.102
3. diabetes: 0.078

Text 3:
1. Migraine: 0.859
2. diabetes: 0.076
3. gastroesophageal reflux disease: 0.022


In [26]:
test_texts = ["severe headache with light sensitivity"]
_, _, probs = learn.predict(test_texts)

In [27]:
probs

tensor([1.8959e-03, 2.3420e-04, 1.5222e-03, 4.8983e-04, 2.7427e-06, 6.7332e-04,
        7.9902e-05, 1.7515e-04, 4.8240e-03, 8.2173e-04, 3.4410e-02, 1.1385e-03,
        5.4918e-03, 5.6351e-01, 1.9929e-01, 3.0771e-03, 2.0716e-05, 1.8753e-04,
        1.3383e-03, 9.1641e-02, 1.4001e-04, 8.1717e-02, 7.2067e-03, 1.0760e-04])

In [28]:
# Then test batch predictions
test_texts = ["severe headache with light sensitivity"]

results = predict_batch_top_3(test_texts, learn)

print("\nBatch predictions:")
for i, preds in enumerate(results):
    print(f"\nText {i+1}:")
    for j, (disease, prob) in enumerate(preds, 1):
        print(f"{j}. {disease}: {prob}")


Batch predictions:

Text 1:
1. Migraine: 0.510
2. Pneumonia: 0.151
3. diabetes: 0.136


In [29]:
# Test with a single prediction first
test_text = "i have a severe running stomach, cold chills, fever, excessive sweating, fatigue and joint pain"

single_pred = get_top_3_predictions(test_text, learn)
print("\nSingle prediction test:")
for i, (disease, prob) in enumerate(single_pred, 1):
    print(f"{i}. {disease}: {prob}")



Single prediction test:
1. Migraine: 0.610
2. Pneumonia: 0.275
3. Malaria: 0.056


## Using the data block API

In [30]:
#dis = DataBlock(
    #blocks=(TextBlock.from_df('text', seq_len=72), CategoryBlock),
    #get_x=ColReader('text'), get_y=ColReader('label'), splitter=ColSplitter())

In [31]:
dis_x = DataBlock(
    blocks=(TextBlock.from_df('text'), CategoryBlock),
    get_x=ColReader('text'),
    get_y=ColReader('label'),
    splitter=RandomSplitter(valid_pct=0.2, seed=42)
)

In [32]:
dls = dis_x.dataloaders(df, bs=64)
dls.show_batch(max_n=2)

,text,category
0,"xxbos xxmaj i 've had this cough that 's been lingering for days and difficulty breathing . xxmaj my fever is xxunk - high , and xxmaj i 'm feeling so weak and tired . xxmaj i 've also been producing a lot of mucus when i cough , and it 's just been so overwhelming to deal with all of these symptoms . xxmaj i 'm getting so xxunk with all of this .",Bronchial Asthma
1,"xxbos xxmaj i 've had a persistent cough for days , and i also have breathing problems . xxmaj i 'm so weak and exhausted , and my fever is through the roof . xxmaj it 's been very difficult to xxunk all of these symptoms , and xxmaj i 've also been coughing up a lot of mucus . xxmaj i 'm becoming so xxunk over everything .",Bronchial Asthma


# ULMFiT approach

In traditional text transfer learning, We use a pre-trained model called a language model. The model we are also going to use in this example was initially trained on Wikipedia on the task of guessing the next word. We then fine-tuned this model for our disease classification task based on symptoms. We can then use this model for our task of disease classification.

But the Wikipedia English might differ from medical jargon, so to further improve our model. We can employ a technique shown in the ULMFIT Paper by Jeremy Howard and Sebastian Ruder. They take the above a step further by fitting the pre-trained model on medical corpus and then using that as a base for our classifier. They noticed that adding this step of training the pretrained model on the task specific corpus gives better result as the model also has better context of the final task.

In [33]:
dls_lm = TextDataLoaders.from_df(symptom_df, path=path, is_lm=True,text_col='text', valid_pct=0.2)

In [34]:
dls_lm.show_batch(max_n=5)

,text,text_
0,"xxbos xxmaj there is a rash particularly bad around my nose , with large red sores that are painful and have yellow rust - colored ooze coming from them . xxbos xxmaj my bloody stools have caused me to lose a lot of things , including iron and bloos . i now have anaemia as a result , and i typically feel rather weak . xxbos xxmaj i 've been experiencing high","xxmaj there is a rash particularly bad around my nose , with large red sores that are painful and have yellow rust - colored ooze coming from them . xxbos xxmaj my bloody stools have caused me to lose a lot of things , including iron and bloos . i now have anaemia as a result , and i typically feel rather weak . xxbos xxmaj i 've been experiencing high fever"
1,"on my work and do n’t feel like doing anything . xxbos xxmaj i 've been experiencing a lot of bowel movement issues recently . xxmaj it 's difficult to go , and it hurts when i do . xxmaj my anus is quite painful , and it has been bleeding whenever i go . xxmaj it 's excruciatingly painful , and xxmaj i 'm quite uneasy . xxbos xxmaj i 'm","my work and do n’t feel like doing anything . xxbos xxmaj i 've been experiencing a lot of bowel movement issues recently . xxmaj it 's difficult to go , and it hurts when i do . xxmaj my anus is quite painful , and it has been bleeding whenever i go . xxmaj it 's excruciatingly painful , and xxmaj i 'm quite uneasy . xxbos xxmaj i 'm having"
2,"well as regular headaches and impaired vision , excessive hunger , a sore neck , depression , irritability , and visual disturbance xxbos xxmaj i 've been experiencing a lot of bowel movement issues recently . xxmaj it 's difficult to go , and it hurts when i do . xxmaj my anus is quite painful , and it has been bleeding whenever i go . xxmaj it 's excruciatingly painful ,","as regular headaches and impaired vision , excessive hunger , a sore neck , depression , irritability , and visual disturbance xxbos xxmaj i 've been experiencing a lot of bowel movement issues recently . xxmaj it 's difficult to go , and it hurts when i do . xxmaj my anus is quite painful , and it has been bleeding whenever i go . xxmaj it 's excruciatingly painful , and"
3,"been battling acid reflux , indigestion , and regular headaches . xxbos xxmaj i 'm coughing nonstop and xxmaj i 'm shivering terribly . i have a stuffy nose and my face is under strain . xxmaj in addition , my throat is coughing up some nasty gunk , and my chest hurts . xxmaj my muscles hurt a lot , and i ca n't smell anything . xxbos i have been","battling acid reflux , indigestion , and regular headaches . xxbos xxmaj i 'm coughing nonstop and xxmaj i 'm shivering terribly . i have a stuffy nose and my face is under strain . xxmaj in addition , my throat is coughing up some nasty gunk , and my chest hurts . xxmaj my muscles hurt a lot , and i ca n't smell anything . xxbos i have been experiencing"
4,"vomit everything i eat , and i have trouble swallowing food because it gets caught in my throat . xxmaj my upper abdomen is bothering me . xxbos xxmaj my skin has a silvery xxunk , particularly on my back , arms and scalp . xxmaj this dusting is xxunk of tiny scales that easily peel off when xxunk . xxbos xxmaj back pain , a persistent cough , and arm and","everything i eat , and i have trouble swallowing food because it gets caught in my throat . xxmaj my upper abdomen is bothering me . xxbos xxmaj my skin has a silvery xxunk , particularly on my back , arms and scalp . xxmaj this dusting is xxunk of tiny scales that easily peel off when xxunk . xxbos xxmaj back pain , a persistent cough , and arm and xxunk"


In [35]:
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()

/opt/conda/lib/python3.10/site-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [36]:
learn.fit_one_cycle(1, 1e-2)

/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.269087,3.649048,0.355830,38.438061,00:01


In [37]:
#learn.save('/kaggle/working/1epoch')

In [38]:
import os

# Create a directory to save the model
os.makedirs('/kaggle/working/models', exist_ok=True)

# Set the model directory for the learner
learn.model_dir = '/kaggle/working/models'

# Now save the model
learn.save('1epoch')

Path('/kaggle/working/models/1epoch.pth')

In [39]:
learn = learn.load('1epoch')

/opt/conda/lib/python3.10/site-packages/fastai/text/learner.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device)


In [40]:
learn.unfreeze()
learn.fit_one_cycle(10, 1e-3)

/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()


epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.681557,3.196092,0.404225,24.436842,00:02
1,3.402061,2.736283,0.434606,15.429528,00:02
2,3.187530,2.484404,0.450159,11.993965,00:02
3,3.002611,2.235703,0.503400,9.353059,00:02
4,2.820902,2.126698,0.519531,8.387123,00:02
5,2.664759,2.037079,0.539931,7.668177,00:02
6,2.536494,1.995208,0.540075,7.353730,00:02
7,2.435808,1.980133,0.545573,7.243703,00:02
8,2.347213,1.953663,0.554977,7.054480,00:02
9,2.272331,1.952309,0.555122,7.044932,00:02


In [41]:
# Now save the model
learn.save_encoder('finetuned')

In [42]:
#learn.save_encoder('finetuned')

In [43]:
TEXT = "I have running nose, stomach and joint pains"
N_WORDS = 40
N_SENTENCES = 2
preds = [learn.predict(TEXT, N_WORDS, temperature=0.75) 
         for _ in range(N_SENTENCES)]

/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is 

/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  self.autocast,self.learn.scaler,self.scales = autocast(dtype=dtype),GradScaler(**self.kwargs),L()
/opt/conda/lib/python3.10/site-packages/fastai/callback/fp16.py:47: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is 

In [44]:
print("\n".join(preds))

i have running nose , stomach and joint pains . My neck hurts , and my neck hurts . i have a high fever and a high fever . My skin has turned yellow , and my skin has turned yellow . Additionally , my
i have running nose , stomach and joint pains . The joints are swollen and swollen , and my chest feels like it 's running . My muscles have been feeling quite weak , and my neck has been extremely stiff . My neck has


In [45]:
dls_clas = TextDataLoaders.from_df(sd_df, path=path,valid='test',text_col='text',label_col='label', text_vocab=dls_lm.vocab)

In [46]:
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)

/opt/conda/lib/python3.10/site-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

In [47]:
from pathlib import Path
learn.path = Path('/kaggle/working')

In [48]:
learn = learn.load_encoder('finetuned')

/opt/conda/lib/python3.10/site-packages/fastai/text/learner.py:135: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(join_path_file(file,self.path/self.model_

In [49]:
len(dls_lm.vocab)

944

In [50]:
learn.fit_one_cycle(1, 2e-2)

epoch,train_loss,valid_loss,accuracy,time
0,2.193524,2.169343,0.479167,00:01


In [51]:
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))

epoch,train_loss,valid_loss,accuracy,time
0,1.405507,1.303245,0.754167,00:01


In [52]:
learn.unfreeze()
learn.fit_one_cycle(2, slice(1e-3/(2.6**4),1e-3))

epoch,train_loss,valid_loss,accuracy,time
0,1.000128,0.908700,0.808333,00:01
1,0.915141,0.761251,0.820833,00:01


In [53]:
learn.predict("Patient presents persistent cough, fever of 101°F for 5 days, and fatigue.")

('Bronchial Asthma',
 tensor(2),
 tensor([0.0088, 0.0027, 0.1409, 0.0323, 0.0062, 0.0185, 0.0275, 0.0049, 0.0169,
         0.0518, 0.0103, 0.0145, 0.0988, 0.0414, 0.0332, 0.1027, 0.0773, 0.0402,
         0.0269, 0.0401, 0.0105, 0.0338, 0.0798, 0.0801]))

Code in one cell


In [54]:
#| code-fold: true
#| code-summary: "Click to see full code in one cell"
#| error: falsepath = Path('/kaggle/input/symptoms-disease-no-id')
path = Path('/kaggle/input/symptoms-disease-no-id')
#symptom_df = pd.read_csv(path_lm/'symptom_synth.csv',index_col=0)
symptom_df = pd.read_csv(path/'symptom_no_id.csv')
sd_df = pd.read_csv(path/'symptom_disease_no_id_col.csv')
#dls_lm = TextDataLoaders.from_df(symptom_df, path=path, is_lm=True, valid_pct=0.2)
dls_lm = TextDataLoaders.from_df(symptom_df, path=path, is_lm=True,text_col='text', valid_pct=0.2)
learn = language_model_learner(dls_lm, AWD_LSTM, metrics=[accuracy, Perplexity()], path=path, wd=0.1).to_fp16()
learn.fit_one_cycle(1, 1e-2)

# Create a directory to save the model
os.makedirs('/kaggle/working/models', exist_ok=True)
# Set the model directory for the learner
learn.model_dir = '/kaggle/working/models'
# Now save the model
learn.save('1epoch')
learn = learn.load('1epoch')
learn.unfreeze()
learn.fit_one_cycle(5, 1e-3)
learn.save_encoder('finetuned')
dls_clas = TextDataLoaders.from_df(sd_df, path=path,valid='test',text_col='text',label_col='label', text_vocab=dls_lm.vocab)
learn = text_classifier_learner(dls_clas, AWD_LSTM, drop_mult=0.5, metrics=accuracy)
from pathlib import Path
learn.path = Path('/kaggle/working')
learn = learn.load_encoder('finetuned')
learn.fit_one_cycle(1, 2e-2)
learn.freeze_to(-2)
learn.fit_one_cycle(1, slice(1e-2/(2.6**4),1e-2))
learn.unfreeze()
learn.fit_one_cycle(5, slice(1e-3/(2.6**4),1e-3))

/opt/conda/lib/python3.10/site-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,4.338441,3.641587,0.350260,38.152348,00:01


/opt/conda/lib/python3.10/site-packages/fastai/text/learner.py:92: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state = torch.load(file, map_location=device)
/opt/conda/lib

epoch,train_loss,valid_loss,accuracy,perplexity,time
0,3.594651,2.960916,0.404514,19.315662,00:02
1,3.266777,2.591931,0.437572,13.355541,00:02
2,3.043697,2.396149,0.468678,10.980807,00:02
3,2.879871,2.304034,0.485388,10.014500,00:02
4,2.751801,2.280301,0.491319,9.779625,00:02


/opt/conda/lib/python3.10/site-packages/fastai/text/learner.py:149: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  wgts = torch.load(wgts_fname, map_location = lambda storage

epoch,train_loss,valid_loss,accuracy,time
0,2.346840,2.431354,0.441667,00:01


epoch,train_loss,valid_loss,accuracy,time
0,1.496796,1.588444,0.737500,00:01


epoch,train_loss,valid_loss,accuracy,time
0,1.061286,1.115787,0.800000,00:01
1,0.974795,0.810600,0.829167,00:01
2,0.882347,0.681513,0.837500,00:01
3,0.803342,0.626208,0.854167,00:01
4,0.753653,0.621115,0.862500,00:01


In [55]:
def get_top_3_predictions(texts, learn):
    """
    Get top 3 predictions for a single text or list of texts
    
    Args:
        texts: Either a single string or a list of strings
        learn: A trained fastai learner for text classification
        
    Returns:
        For a single text: List of (label, probability) tuples
        For multiple texts: List of lists of (label, probability) tuples
    """
    # Handle both single text and list of texts
    is_single = isinstance(texts, str)
    if is_single:
        texts = [texts]
    
    disease_vocab = learn.dls.vocab[1]
    n_classes = len(disease_vocab)
    
    # Try to use DataLoader for batch prediction if model supports it
    try:
        # This is more efficient but might not work with all models
        preds = learn.get_preds(dl=learn.dls.test_dl(texts))
        probs_list = preds[0]  # Tensor of shape [batch_size, n_classes]
        
        all_predictions = []
        for probs in probs_list:
            n_preds = min(3, n_classes)
            top_k_indices = probs.argsort(descending=True)[:n_preds]
            
            predictions = []
            for idx in top_k_indices:
                label = disease_vocab[int(idx)]
                probability = float(probs[idx])
                predictions.append((label, probability))
            
            all_predictions.append(predictions)
    
    except Exception:
        # Fall back to individual prediction if batch method fails
        all_predictions = []
        for text in texts:
            _, _, probs = learn.predict(text)
            
            n_preds = min(3, n_classes)
            top_k_indices = probs.argsort(descending=True)[:n_preds]
            
            predictions = []
            for idx in top_k_indices:
                label = disease_vocab[int(idx)]
                probability = float(probs[idx])
                predictions.append((label, probability))
            
            all_predictions.append(predictions)
    
    return all_predictions[0] if is_single else all_predictions


def display_predictions(predictions, texts=None):
    """
    Display formatted predictions
    
    Args:
        predictions: Either a list of (label, prob) tuples or a list of such lists
        texts: Optional list of input texts to display with predictions
    """
    # If predictions is a list of (label, prob) tuples (single text case)
    if isinstance(predictions[0], tuple):
        for i, (disease, prob) in enumerate(predictions, 1):
            print(f"{i}. {disease}: {prob:.3f}")
    # If predictions is a list of lists (batch case)
    else:
        for i, preds in enumerate(predictions):
            if texts:
                print(f"\nText: {texts[i][:50]}...")
            else:
                print(f"\nSample #{i+1}:")
            for j, (disease, prob) in enumerate(preds, 1):
                print(f"  {j}. {disease}: {prob:.3f}")

In [56]:
# Assuming 'learn' is your trained FastAI model

# Example 1: Single input
single_text = "Patient presents with persistent cough, fever of 101°F for 5 days, and fatigue."
single_result = get_top_3_predictions(single_text, learn)

print("SINGLE TEXT PREDICTION:")
print(f"Input: {single_text}")
print("Top 3 predictions:")
display_predictions(single_result)


# Example 2: Batch input (small batch)
batch_texts = [
    "Patient presents with persistent cough, fever of 101°F for 5 days, and fatigue.",
    "7-year-old with red, itchy rash on face and arms, started 2 days after camping trip.",
    "Adult male with sudden onset of severe headache, described as 'worst headache of my life'.",
    "Patient reports joint pain in fingers and wrists, worse in the morning, accompanied by stiffness."
]
batch_results = get_top_3_predictions(batch_texts, learn)

print("\nBATCH PREDICTION EXAMPLE:")
display_predictions(batch_results, batch_texts)


# Example 3: Processing a medium-sized dataset
medium_dataset = [
    f"Patient {i}: Symptoms include {symptom}" for i, symptom in enumerate([
        "fever and sore throat",
        "chest pain radiating to left arm",
        "swollen lymph nodes and night sweats",
        "difficulty breathing and wheezing",
        "abdominal pain and vomiting",
        "frequent urination and excessive thirst",
        "joint pain and morning stiffness",
        "persistent headache and blurred vision",
        "unexplained weight loss and fatigue",
        "skin rash and itching"
    ] * 3)  # Repeat symptoms to create 30 samples
]

print("\nPROCESSING MEDIUM DATASET:")
medium_results = get_top_3_predictions(medium_dataset, learn)
# Display first 3 results only for brevity
print("First 3 results from medium dataset:")
display_predictions(medium_results[:3], medium_dataset[:3])


# Example 4: Working with DataFrame data
# This example demonstrates how you might use the function with pandas DataFrame
import pandas as pd

# Create a sample DataFrame
df = pd.DataFrame({
    'patient_id': range(1001, 1006),
    'age': [45, 12, 67, 32, 54],
    'gender': ['M', 'F', 'M', 'F', 'M'],
    'symptoms': [
        "Persistent dry cough and fever for 3 days",
        "Skin rash with small fluid-filled blisters, mild fever",
        "Shortness of breath, chest tightness, wheezing when exercising",
        "Severe migraine, sensitivity to light, nausea",
        "Pain and swelling in the right knee, difficulty walking"
    ]
})

print("\nPROCESSING DATAFRAME:")
print("Sample DataFrame:")
print(df[['patient_id', 'symptoms']].head())

# Process the symptoms column
df_results = get_top_3_predictions(df['symptoms'].tolist(), learn)

# Add predictions back to the DataFrame
df['top_prediction'] = [pred[0][0] for pred in df_results]  # First prediction label
df['confidence'] = [pred[0][1] for pred in df_results]      # First prediction probability

print("\nDataFrame with predictions:")
print(df[['patient_id', 'symptoms', 'top_prediction', 'confidence']])

SINGLE TEXT PREDICTION:
Input: Patient presents with persistent cough, fever of 101°F for 5 days, and fatigue.
Top 3 predictions:
1. Malaria: 0.212
2. diabetes: 0.173
3. Pneumonia: 0.111



BATCH PREDICTION EXAMPLE:

Text: Patient presents with persistent cough, fever of 1...
  1. Malaria: 0.212
  2. diabetes: 0.173
  3. Pneumonia: 0.111

Text: 7-year-old with red, itchy rash on face and arms, ...
  1. Impetigo: 0.548
  2. Psoriasis: 0.179
  3. Fungal infection: 0.058

Text: Adult male with sudden onset of severe headache, d...
  1. Malaria: 0.171
  2. Dengue: 0.105
  3. diabetes: 0.088

Text: Patient reports joint pain in fingers and wrists, ...
  1. Psoriasis: 0.320
  2. diabetes: 0.144
  3. Dengue: 0.089

PROCESSING MEDIUM DATASET:


First 3 results from medium dataset:

Text: Patient 0: Symptoms include fever and sore throat...
  1. gastroesophageal reflux disease: 0.204
  2. diabetes: 0.162
  3. Malaria: 0.117

Text: Patient 1: Symptoms include chest pain radiating t...
  1. gastroesophageal reflux disease: 0.308
  2. Hypertension: 0.205
  3. Malaria: 0.082

Text: Patient 2: Symptoms include swollen lymph nodes an...
  1. Impetigo: 0.264
  2. Dengue: 0.119
  3. diabetes: 0.069

PROCESSING DATAFRAME:
Sample DataFrame:
   patient_id                                                        symptoms
0        1001                       Persistent dry cough and fever for 3 days
1        1002          Skin rash with small fluid-filled blisters, mild fever
2        1003  Shortness of breath, chest tightness, wheezing when exercising
3        1004                   Severe migraine, sensitivity to light, nausea
4        1005         Pain and swelling in the right knee, difficulty walking



DataFrame with predictions:
   patient_id                                                        symptoms  \
0        1001                       Persistent dry cough and fever for 3 days   
1        1002          Skin rash with small fluid-filled blisters, mild fever   
2        1003  Shortness of breath, chest tightness, wheezing when exercising   
3        1004                   Severe migraine, sensitivity to light, nausea   
4        1005         Pain and swelling in the right knee, difficulty walking   

  top_prediction  confidence  
0       diabetes    0.469087  
1       Impetigo    0.218025  
2        allergy    0.364993  
3       diabetes    0.684080  
4       diabetes    0.261741  
